In [1]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time

In [2]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}


In [3]:
def get_links(year, month:str, page):
    url ='http://www.arthub.co.kr/sub01/board033_list.htm?'
    seed = 'http://www.arthub.co.kr/sub01/'
    
    params = {
        'k_date':year,
        'k_month':month,
        'PHPSESSID':'e124fc854822909d9ec3618c1e6089f2',
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('a.sub01_list_title')
    
    return [(str(year)+month,a.get_text(),seed+a['href']) for a in aList]

In [4]:
get_links(2018,'09',1)

[('201809',
  '나의 예술, 나의 인생 THE ARTISTS IN GANGWON :: Photography',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26937'),
 ('201809',
  '이재건展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25924'),
 ('201809',
  'An unfamiliar gaze - 정의철展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26876'),
 ('201809',
  '조성희展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=25935'),
 ('201809',
  '서근범展 :: Painting',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26893'),
 ('201809',
  '쪽방프로젝트 - 육효진展 :: Installation',
  'http://www.arthub.co.kr/sub01/board03_view.htm?No=26917')]

html 태그가 매우 드럽게 달려있다

부적절한 tree 구조인데다가 identification 규칙이 일정치 않음, 일일이 찾기힘듦

전시정보를 가져오는 함수 get_info를 짰는데 next.next.next... 이런식으로 드럽다, 어쩔수없었다

In [6]:
def get_info(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    t3 = soup.select('table')[3]

    info = []
    for el in [el.next.next.next.next.next for el in t3.find_all('td',{'width':'84','height':'18','align':'left'})]: #전시정보 뽑아오기, tag가 드러움
        if el.string==None: #class 'bs4.element.Tag'
            info.append(el.text)
        else:                          #class 'bs4.element.NavigableString'
            info.append(el.string) 

    artists = list(map(lambda s:s.strip(), info[0].split(',')))
    articles = [td.next['href'] for td in t3.find_all('td',{'width':'510'})[1:]]

    del info[0]
    info.insert(0,artists)
    info.append(articles)
    
    return info

In [7]:
get_info('http://www.arthub.co.kr/sub01/board03_view.htm?No=26567')

[['강지만',
  '김경섭',
  '김용석',
  '김아영',
  '문기전',
  '박정원',
  '박정용',
  '박흥선',
  '성태진',
  '송지연',
  '윤기원',
  '이수정',
  '이재열',
  '진선미'],
 '2018-08-01 ▶ 2018-08-04',
 '없음',
 'Open 14:00 ~ Close 20:00',
 '아트팩토리 후(ART FACTORY hoo)\xa0 다른전시 보기',
 '강원도 원주시 문막읍 비야동길 10-12 (후용리 1318) 나동창고 ',
 '010-4014-7488',
 'www.facebook.com/artfactoryhoo',
 ['http://www.newsis.com/view/?id=NISX20180710_0000359747&cid=10701']]

텍스트를 가져오는 함수 get_text

In [8]:
def get_text(url):
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    ps = [p.text for p in soup.find_all('p',{'align':'justify'})]
    text = ''.join(ps)
    
    return text.strip()

In [9]:
get_text('http://www.arthub.co.kr/sub01/board03_view.htm?No=19282')

'The birth of Life\n        물은 모든 생명의 원천이다. 모든 사물은 물에서 시작되어 그 끝 역시 물로 마무리 되고 그 과정은 지속적인 순환과정을 겪는다. \n        그리고 물의 순환과정을 통해 우리는 생명의 강인함과 진리를 깨닫게 된다. 또한 물에서 비롯된 생명은 하나가 모여 전체가 되듯이 \n        한 화면 속에서 개개의 소재들이 모여 하나의 화면을 구성하며 전일적인 개체로 성장한다.\n\n\n        The song of Life\n        모든 생명은 아름답고 작가의 작업은 그 아름다움을 노래하기 위한 수단이다. 그리고 그 아름다움을 만들어내는 모든 생명의 기원은 \n        물이다. 그래서 그의 작품 속에는 물을 상징하는 소재들이 그려져 있다. 또한 물위에서 기원하듯 그 주위에는 생명력을 품은 아름다운 \n        꽃들이 둘러싸며 저마다의 생명력을 뿜어내고 있다.The \n        energy of Life\n        생명은 항상 강인함을 지닌다. 어떠한 시련에도 생명의 순환은 결코 멈추지 않는다. 그리고 이것은 생명의 원천인 물이 가지는 에너지이기도 \n        하다. 작가는 이러한 끊임없는 순환의 과정 속의 생명이 가지는 강인함과 아름다움을 노래하기 위해 캔버스 위를 유화와 아크릴을 \n        통해 다채로운 색채로 표현한다. 잔잔한 한 폭의 정물화 인듯하지만 화려하고 과감한 색채의 대비는 우리에게 생명의 에너지를 속삭인다. \n         ⓒ'

샘플 데이터를 만들어보자, 55개의 전시만 가져왔다

In [15]:
start = time.time()

arts = []
for y in range(2013,2014):
    for m in tqdm(range(1,2)):
        if m<10:
            m = '0'+str(m)
        else:
            m = str(m)
        m_links = []
        for p in range(1,6): 
            try:
                for _ in range(2): #TimeoutError 방지위해 두번까지 시도한다
                    m_links.extend(get_links(y,m,p))
                    break
            except Exception as e:
                print(e)
                time.sleep(1)
                continue
                
        m_links = list(set(m_links)) #중복 링크 제거
        for link in m_links:
            try:
                info = get_info(link[2])
                text = get_text(link[2])

                temp = [*link,*info,text]
                arts.append(temp)
                print(link)
            except Exception as e:
                print(e)
                continue
                
end = time.time()
data = pd.DataFrame(arts,columns=['연월','전시제목','링크','전시작가','전시일정','초대일시','관람시간','전시장소','주소','연락처','홈페이지','아티클','텍스트'])
data.sample(5)

print(end-start)





  0%|          | 0/1 [00:00<?, ?it/s]

('201301', '카페림보 - 김한민展 :: Exhibit Theater', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10924')
('201301', '박명희展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10603')
('201301', '잘 어질러진 방 : my place - 유소라展 :: Quilt', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10942')
('201301', '부드러운 교란-백남준을 말하다 :: Media Art', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10964')
('201301', 'space-scape :: Various', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10755')
('201301', '꽃들의 외출 - 황규태展 :: Photography', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=11129')
('201301', 'Fashion과 Passion사이 :: Various', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10570')
('201301', 'Beyond - 서옥석展 :: Intermedia', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10641')
('201301', '김지혜展 :: Intermedia', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10614')
('201301', 'Maden Pictures 13 :: Photography & Video', 'http://www.arthub.co.kr/sub01/board03_view.h

('201301', 'THE SYMBOL - 도병규展 :: Photography', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10821')
('201301', '의미 불명 - 구현회展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10784')
('201301', '이나경展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10770')
('201301', '시간을 뒹구는 돌 - 전형근展 :: Photography', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10847')
('201301', '나무에 꿈을 새기다 - 오석규展 :: Sculpture', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=11000')
('201301', '그림 • 가구 예술향유 :: Painting & Interior', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10839')
('201301', '손수용展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10609')
('201301', '소리없이 내리는 비 :: Various', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=11026')
('201301', '보자기, 마음의 기하학展 :: Folk Art', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10710')
('201301', '히말라야 유랑 - 이훈구展 :: Photography', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10970')
(

('201301', 'FRAGILE - 김지훈展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10694')
('201301', '交感 - 최은옥展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10992')
('201301', '원더러스트: 또다른 언덕 너머로 가는 끊임없는 여정 :: Various', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=11041')
('201301', '경계를 넘어서 Beyond Boundaries II :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10666')
('201301', '죽은 새의 눈을 가진 남자 - 려은展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10731')
('201301', '박물관 Image展 :: Various', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10780')
('201301', '경계에 선 풍경 - 김미강展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=11034')
('201301', 'MOTION - 신상호展 :: Animation', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10757')
('201301', 'Grapes - 김대연展 :: Painting', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10988')
('201301', '셋: 이화정, 서림, 이정호展 :: Painting', 'http://www.arthub.co.kr/sub01/boa





100%|██████████| 1/1 [02:53<00:00, 173.14s/it]





('201301', 'Lost in Translation - Bang & Lee展 :: Video', 'http://www.arthub.co.kr/sub01/board03_view.htm?No=10751')
173.1508297920227


In [2]:
len(data.전시제목.unique())

In [16]:
data[data.duplicated(subset='전시제목')]

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트


In [40]:
data

,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
0,201301,카페림보 - 김한민展 :: Exhibit Theater,http://www.arthub.co.kr/sub01/board03_view.htm...,[김한민(Kim Hanmin)],2013-01-02 ▶ 2013-02-27,2013-02-15 / 16,Open 11:00 ~ Close 23:30,테이크아웃드로잉(Takeout Drawing) 이태원 다른전시 보기,서울시 용산구 이태원동 637,02-790-2637,www.takeoutdrawing.com,[],● 카페림보 - 김한민展\n 생의 의미와 목적 그리고 사회에 대한 소속...
1,201301,박명희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,[박명희(Park Myunghee)],2013-01-15 ▶ 2013-01-27,없음,Open 9:00 ~ Close 20:00,세종갤러리(SEJONG GALLERY) 다른전시 보기,서울시 중구 충무로2가 61-3 세종호텔,02-3705-9021,www.sejonggallery.co.kr,[],● 박명희展\n ‘꿈과 자연’이라는 주제로 시각적인 형상과 상상의 나래...
2,201301,잘 어질러진 방 : my place - 유소라展 :: Quilt,http://www.arthub.co.kr/sub01/board03_view.htm...,[유소라(Yu Sora)],2013-01-30 ▶ 2013-02-19,2013-01-30 PM 5:00,Open 10:00 ~ Close 18:00(일요일 휴관),갤러리41(Gallery41) 다른전시 보기,서울시 종로구 사간동 41,02-744-0341,,[],● 잘 어질러진 방 : my place\n - 소소한 욕심들로 채워...
3,201301,부드러운 교란-백남준을 말하다 :: Media Art,http://www.arthub.co.kr/sub01/board03_view.htm...,"[백남준, 저드 얄커트, 만프레드 레베, 샬럿 무어만]",2013-01-29 ▶ 2013-06-30,2013-02-07 PM 5:00,"Open 10:00 ~ Close 18:00(2주, 4주 월요일 휴관)",백남준아트센터(NJP ART CENTER) 다른전시 보기,경기도 용인시 기흥구 상갈동 85,031-201-8529,www.njpartcenter.kr,[],● 부드러운 교란-백남준을 말하다\n 백남준아트센터의 2013년도 첫 ...
4,201301,space-scape :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"[김민정, 김병주, 김용관]",2013-01-05 ▶ 2013-01-28,,Open 10:30 ~ Close 20:00(주말 20:30),롯데갤러리 일산점 다른전시 보기,경기도 고양시 일산동구 장항동 784 롯데백화점 일산점 B1,031-909-2688,,[],● 세 가지 공간 그리고 하나의 풍경\n 저희 롯데갤러리 일산점에서는 ...
5,201301,꽃들의 외출 - 황규태展 :: Photography,http://www.arthub.co.kr/sub01/board03_view.htm...,[황규태(Hwang Kyutae)],2013-01-23 ▶ 2013-03-03,없음,Open 10:30 ~ Close 18:30(백화점 휴점일 휴관),신세계백화점 본점 신세계갤러리 다른전시 보기,서울시 중구 충무로 1가 52-1,02-310-1924,,[],"● 꽃들의 외출 - 황규태展\n 황규태, 한국현대사진의 아방가르드\n\..."
6,201301,Fashion과 Passion사이 :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"[정유민, 최유주, 김현영]",2013-01-07 ▶ 2013-01-30,없음,Open 11:00 ~ Close 20:00,드로잉 11(drawing 11) 다른전시 보기,서울시 강남구 신사동 545-11,02-511-2930,blog.naver.com/drawing114u,[],● Fashion과 Passion사이\n 드로잉11은 어반아트가 젊은 ...
7,201301,Beyond - 서옥석展 :: Intermedia,http://www.arthub.co.kr/sub01/board03_view.htm...,[서옥석(Ok Seo 徐沃錫)],2013-01-11 ▶ 2013-01-18,없음,Open 10:00 ~ Close 18:00,Galerie Arteconte 다른전시 보기,"Paris, France",seok2121@hanmail.net,,[],● Beyond\n 본 전시회의 사진들은 나의 정신적 판타지의 세계를 ...
8,201301,김지혜展 :: Intermedia,http://www.arthub.co.kr/sub01/board03_view.htm...,[김지혜(Kim Jihea)],2013-01-09 ▶ 2013-01-15,없음,Open 10:00 ~ Close 19:00,갤러리 라메르(gallery La Mer) 다른전시 보기,서울시 종로구 인사동 194,02-730-5454,www.gallerylamer.com,[],"● 김지혜의 사진\n - 도시생태학, 도시를 스캔하는 이미지 헌터\..."
9,201301,Maden Pictures 13 :: Photography & Video,http://www.arthub.co.kr/sub01/board03_view.htm...,"[곽현진, 김아영, 박현두, 오용석, 장성은]",2013-01-15 ▶ 2013-03-03,없음,Open 11:00 ~ Close 19:00(월요일 휴관),아라리오 갤러리 서울(ARARIO GALLERY SEOUL) 청담 다른전시 보기,서울시 강남구 청담동 99-5,02-723-6190,www.arariogallery.co.kr,[],● Maden Pictures 13\n 아라리오 갤러리 청담은 2013...


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 13 columns):
연월      181 non-null object
전시제목    181 non-null object
링크      181 non-null object
전시작가    181 non-null object
전시일정    181 non-null object
초대일시    181 non-null object
관람시간    181 non-null object
전시장소    181 non-null object
주소      181 non-null object
연락처     181 non-null object
홈페이지    181 non-null object
아티클     181 non-null object
텍스트     181 non-null object
dtypes: object(13)
memory usage: 18.5+ KB


In [35]:
data.to_csv('/Users/JEONG/Desktop/test2.csv')

In [36]:
d = pd.read_csv('/Users/JEONG/Desktop/test2.csv')

In [37]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 14 columns):
Unnamed: 0    181 non-null int64
연월            181 non-null int64
전시제목          181 non-null object
링크            181 non-null object
전시작가          181 non-null object
전시일정          181 non-null object
초대일시          177 non-null object
관람시간          181 non-null object
전시장소          181 non-null object
주소            181 non-null object
연락처           181 non-null object
홈페이지          132 non-null object
아티클           181 non-null object
텍스트           177 non-null object
dtypes: int64(2), object(12)
memory usage: 19.9+ KB


In [38]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181 entries, 0 to 180
Data columns (total 14 columns):
Unnamed: 0    181 non-null int64
연월            181 non-null int64
전시제목          181 non-null object
링크            181 non-null object
전시작가          181 non-null object
전시일정          181 non-null object
초대일시          177 non-null object
관람시간          181 non-null object
전시장소          181 non-null object
주소            181 non-null object
연락처           181 non-null object
홈페이지          132 non-null object
아티클           181 non-null object
텍스트           177 non-null object
dtypes: int64(2), object(12)
memory usage: 19.9+ KB


In [39]:
d

,Unnamed: 0,연월,전시제목,링크,전시작가,전시일정,초대일시,관람시간,전시장소,주소,연락처,홈페이지,아티클,텍스트
0,0,201301,카페림보 - 김한민展 :: Exhibit Theater,http://www.arthub.co.kr/sub01/board03_view.htm...,['김한민(Kim Hanmin)'],2013-01-02 ▶ 2013-02-27,2013-02-15 / 16,Open 11:00 ~ Close 23:30,테이크아웃드로잉(Takeout Drawing) 이태원 다른전시 보기,서울시 용산구 이태원동 637,02-790-2637,www.takeoutdrawing.com,[''],● 카페림보 - 김한민展\n 생의 의미와 목적 그리고 사회에 대한 소속...
1,1,201301,박명희展 :: Painting,http://www.arthub.co.kr/sub01/board03_view.htm...,['박명희(Park Myunghee)'],2013-01-15 ▶ 2013-01-27,없음,Open 9:00 ~ Close 20:00,세종갤러리(SEJONG GALLERY) 다른전시 보기,서울시 중구 충무로2가 61-3 세종호텔,02-3705-9021,www.sejonggallery.co.kr,[''],● 박명희展\n ‘꿈과 자연’이라는 주제로 시각적인 형상과 상상의 나래...
2,2,201301,잘 어질러진 방 : my place - 유소라展 :: Quilt,http://www.arthub.co.kr/sub01/board03_view.htm...,['유소라(Yu Sora)'],2013-01-30 ▶ 2013-02-19,2013-01-30 PM 5:00,Open 10:00 ~ Close 18:00(일요일 휴관),갤러리41(Gallery41) 다른전시 보기,서울시 종로구 사간동 41,02-744-0341,NaN,[''],● 잘 어질러진 방 : my place\n - 소소한 욕심들로 채워...
3,3,201301,부드러운 교란-백남준을 말하다 :: Media Art,http://www.arthub.co.kr/sub01/board03_view.htm...,"['백남준', '저드 얄커트', '만프레드 레베', '샬럿 무어만']",2013-01-29 ▶ 2013-06-30,2013-02-07 PM 5:00,"Open 10:00 ~ Close 18:00(2주, 4주 월요일 휴관)",백남준아트센터(NJP ART CENTER) 다른전시 보기,경기도 용인시 기흥구 상갈동 85,031-201-8529,www.njpartcenter.kr,[''],● 부드러운 교란-백남준을 말하다\n 백남준아트센터의 2013년도 첫 ...
4,4,201301,space-scape :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"['김민정', '김병주', '김용관']",2013-01-05 ▶ 2013-01-28,NaN,Open 10:30 ~ Close 20:00(주말 20:30),롯데갤러리 일산점 다른전시 보기,경기도 고양시 일산동구 장항동 784 롯데백화점 일산점 B1,031-909-2688,NaN,[''],● 세 가지 공간 그리고 하나의 풍경\n 저희 롯데갤러리 일산점에서는 ...
5,5,201301,꽃들의 외출 - 황규태展 :: Photography,http://www.arthub.co.kr/sub01/board03_view.htm...,['황규태(Hwang Kyutae)'],2013-01-23 ▶ 2013-03-03,없음,Open 10:30 ~ Close 18:30(백화점 휴점일 휴관),신세계백화점 본점 신세계갤러리 다른전시 보기,서울시 중구 충무로 1가 52-1,02-310-1924,NaN,[''],"● 꽃들의 외출 - 황규태展\n 황규태, 한국현대사진의 아방가르드\n\..."
6,6,201301,Fashion과 Passion사이 :: Various,http://www.arthub.co.kr/sub01/board03_view.htm...,"['정유민', '최유주', '김현영']",2013-01-07 ▶ 2013-01-30,없음,Open 11:00 ~ Close 20:00,드로잉 11(drawing 11) 다른전시 보기,서울시 강남구 신사동 545-11,02-511-2930,blog.naver.com/drawing114u,[''],● Fashion과 Passion사이\n 드로잉11은 어반아트가 젊은 ...
7,7,201301,Beyond - 서옥석展 :: Intermedia,http://www.arthub.co.kr/sub01/board03_view.htm...,['서옥석(Ok Seo 徐沃錫)'],2013-01-11 ▶ 2013-01-18,없음,Open 10:00 ~ Close 18:00,Galerie Arteconte 다른전시 보기,"Paris, France",seok2121@hanmail.net,NaN,[''],● Beyond\n 본 전시회의 사진들은 나의 정신적 판타지의 세계를 ...
8,8,201301,김지혜展 :: Intermedia,http://www.arthub.co.kr/sub01/board03_view.htm...,['김지혜(Kim Jihea)'],2013-01-09 ▶ 2013-01-15,없음,Open 10:00 ~ Close 19:00,갤러리 라메르(gallery La Mer) 다른전시 보기,서울시 종로구 인사동 194,02-730-5454,www.gallerylamer.com,[''],"● 김지혜의 사진\n - 도시생태학, 도시를 스캔하는 이미지 헌터\..."
9,9,201301,Maden Pictures 13 :: Photography & Video,http://www.arthub.co.kr/sub01/board03_view.htm...,"['곽현진', '김아영', '박현두', '오용석', '장성은']",2013-01-15 ▶ 2013-03-03,없음,Open 11:00 ~ Close 19:00(월요일 휴관),아라리오 갤러리 서울(ARARIO GALLERY SEOUL) 청담 다른전시 보기,서울시 강남구 청담동 99-5,02-723-6190,www.arariogallery.co.kr,[''],● Maden Pictures 13\n 아라리오 갤러리 청담은 2013...


In [19]:
corpus = ''
for txt in data.텍스트:
    if txt!=None:
        corpus +=txt

In [20]:
# corpus #텍스트를 합쳐 코퍼스를 만든다
len(corpus)

297474

아래부터는 텍스트 전처리 과정

In [21]:
import nltk
from konlpy.tag import Kkma,Twitter
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize, regexp_tokenize
from matplotlib import font_manager, rc

In [22]:
kkma = Kkma()
# twitter = Twitter()

In [24]:
tokens = kkma.morphs(corpus)

In [25]:
word_tokenize(corpus)[:15]

['●',
 '카페림보',
 '-',
 '김한민展',
 '생의',
 '의미와',
 '목적',
 '그리고',
 '사회에',
 '대한',
 '소속감을',
 '철저히',
 '잃어버린',
 '6명',
 '‘']

In [26]:
count2 = Counter(tokens)
count2.most_common(30) #불용어 처리를 하지않은 결과이다

[('하', 5657),
 ('ㄴ', 4998),
 ('는', 4747),
 ('의', 4651),
 ('이', 4567),
 ('.', 3895),
 ('을', 3758),
 (',', 2522),
 ('다', 2245),
 ('은', 2221),
 ('어', 2159),
 ('에', 2143),
 ('고', 1887),
 ('를', 1791),
 ('있', 1648),
 ('되', 1411),
 ('들', 1307),
 ('ㄴ다', 1224),
 ('과', 1194),
 ('으로', 1193),
 ('에서', 1088),
 ('ㄹ', 1024),
 ('것', 991),
 ('가', 978),
 ('그', 883),
 ('게', 814),
 ('로', 813),
 ('었', 788),
 ('작가', 744),
 ('작품', 653)]

In [27]:
text = nltk.Text(tokens)

In [28]:
text.similar("작가")

그 우리 나 인간 회화 작품 이미지 자신 하나 전시 도시 존재 세계 사람 시간 나무 주제 그림 삶 이야기


In [29]:
text.similar("존재")

작가 전시 이미지 의미 그 변화 이야기 세계 회화 작업 소재 표현 형태 나 하나 생각 다양 에너지 행위 탐구


In [30]:
text.similar("희망")

작품 것 삶 현실 작업 공간 시간 세상 사진 인간 그림 사랑 생각 풍경 느낌 아름다움 시선 형식 경험 꿈
